# Script de generación de dataset
---

* Script de Python para probar una página web según seis criterios de accesibilidad.
* Estos criterios se basan en un informe de WebAIM y se implementan siguiendo las Pautas de Accesibilidad al Contenido Web 2.2 (WCAG 2.2):
    * Idioma del documento faltante
    * Faltan textos alternativos para las imágenes
    * Etiquetas de entrada de formulario faltantes
    * Botones vacíos
    * Enlaces vacíos
    * Texto de bajo contraste

## Puntuación

* El nivel de accesibilidad requerido es un valor entre 0 y 1.
* Se compara con el nivel de accesibilidad real alcanzado en la prueba, que se calcula dividiendo las comprobaciones exitosas entre el número total de comprobaciones ejecutadas.
* Si el nivel de accesibilidad alcanzado es igual o superior al requerido, el programa devolverá **sin_prob_aw**; de lo contrario, **con_prob_aw**.

In [66]:
class Utils:

    # Calcula el xpath de un elemento
    @staticmethod
    def xpath_soup(element):
        if element is None:
            return '/html'
        components = []
        child = element if element.name else element.parent
        for parent in child.parents:  # type: bs4.element.Tag
            siblings = parent.find_all(child.name, recursive=False)
            components.append(
                child.name if 1 == len(siblings) else '%s[%d]' % (
                    child.name,
                    next(i for i, s in enumerate(siblings, 1) if s is child)
                    )
                )
            child = parent
        components.reverse()
        if not components:
            return '/html'
        return '/%s' % '/'.join(components)

    # Extrae todos los textos de una página
    @staticmethod
    def extract_texts(soup):
        soup2 = soup
    
        # elimina script, style y title
        for invisible_element in soup2(["script", "style", "title", "noscript"]):
            invisible_element.extract()
    
        # elimina comentarios
        comments = soup2.findAll(string=lambda text:isinstance(text, Comment))
        for comment in comments:
            comment.extract()
    
        # elimina doctype
        doctype = soup2.find(string=lambda text:isinstance(text, Doctype))
        if not doctype is None:
            doctype.extract()
    
        # obtiene todos los elementos con texto
        texts = []
        texts_on_page = soup2.findAll(string=True)
        for text in texts_on_page:
            if not text.strip() == "" and not text == "\n":
                texts.append(text.parent)
    
        return texts

    # Devuelve el color de un texto
    @staticmethod
    def get_background_color(driver, text):
        if text is None:
            return "rgba(255,255,255,1)"
    
        selenium_element = driver.find_element(by="xpath", value=Utils.xpath_soup(text))
        background_color = Utils.convert_to_rgba_value(selenium_element.value_of_css_property('background-color'))
    
        if eval(background_color[4:])[3] == 0:
            return Utils.get_background_color(driver, text.parent)
    
        return background_color

    # Convierte un color a formato rgba
    @staticmethod
    def convert_to_rgba_value(color):
        if color[:4] != "rgba":
            rgba_tuple = eval(color[3:]) + (1,)
            color = "rgba" + str(rgba_tuple)
        return color

    # Calcula la relación de contraste entre el color del texto y el color de fondo  
    @staticmethod
    def get_contrast_ratio(text_color, background_color):
        # preparando los valores RGB
        r_text = Utils.convert_rgb_8bit_value(text_color[0])
        g_text = Utils.convert_rgb_8bit_value(text_color[1])
        b_text = Utils.convert_rgb_8bit_value(text_color[2])
        r_background = Utils.convert_rgb_8bit_value(background_color[0])
        g_background = Utils.convert_rgb_8bit_value(background_color[1])
        b_background = Utils.convert_rgb_8bit_value(background_color[2])
    
        # cálculo de la luminancia relativa
        luminance_text = 0.2126 * r_text + 0.7152 * g_text + 0.0722 * b_text
        luminance_background = 0.2126 * r_background + 0.7152 * g_background + 0.0722 * b_background
    
        # comprueba si luminance_text o luminance_background son más claros
        if luminance_text > luminance_background:
            # calcular la relación de contraste cuando luminance_text es la luminancia relativa del color más claro
            contrast_ratio = (luminance_text + 0.05) / (luminance_background + 0.05)
        else:
            # calcula la relación de contraste cuando luminance_background es la luminancia relativa del color más claro
            contrast_ratio = (luminance_background + 0.05) / (luminance_text + 0.05)
    
        return contrast_ratio

    # Convierte un valor rgb al formato necesario
    @staticmethod
    def convert_rgb_8bit_value(single_rgb_8bit_value):
        # divide el valor de 8-bit entre 255
        srgb = single_rgb_8bit_value / 255
    
        # comprueba si el valor de srgb es menor o igual a 0.03928
        if srgb <= 0.03928:
            return srgb / 12.92
    
        return ((srgb + 0.055) / 1.055) ** 2.4

In [67]:
import sys
import urllib.parse
from bs4 import BeautifulSoup, Comment, Doctype
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
import pandas as pd

In [68]:
class TAW:
  
    def __init__(self, nombre, url):
        self.nombre = nombre
        self.url = url
        self.required_degree = 0.75
        options = ChromeOptions()
        options.headless = True
        options.add_argument("--log-level=3")
        self.driver = webdriver.Chrome(options=options)
        self.driver.get(self.url)
        self.page = BeautifulSoup(self.driver.page_source, "html.parser")
        self.correct = {"doc_language":0, "alt_texts":0, "input_labels":0, "empty_buttons":0, "empty_links":0, "color_contrast":0}
        self.wrong = {"doc_language":0, "alt_texts":0, "input_labels":0, "empty_buttons":0, "empty_links":0, "color_contrast":0}

    # Ejecuta las pruebas de la página actual
    def test_page(self):
        self.check_doc_language()
        self.check_alt_texts()
        self.check_input_labels()
        self.check_buttons()
        self.check_links()
        self.check_color_contrast()
        self.driver.quit()
        return self.calculate_result()

    # Verifica si el idioma del documento esta configurado (3.1.1 H57)
    def check_doc_language(self):
         # verifica si el atributo lang existe y no está vacío
        lang_attr = self.page.find("html").get_attribute_list("lang")[0]
        if not lang_attr is None and not lang_attr == "":
            self.correct["doc_language"] += 1
        elif not lang_attr is None:
            self.wrong["doc_language"] += 1
        else:
            self.wrong["doc_language"] += 1
       
    # Verifica si todas las imágenes de la página tienen un texto alternativo (1.1.1 H37)
    def check_alt_texts(self):
        # obtiene todos los elementos img
        img_elements = self.page.find_all("img")
        for img_element in img_elements:
            # verifica si el elemento img tiene un texto alternativo que no esté vacío
            alt_text = img_element.get_attribute_list('alt')[0]
            if not alt_text is None and not alt_text == "":
                self.correct["alt_texts"] += 1
            elif not alt_text is None:
                self.wrong["alt_texts"] += 1
            else:
                self.wrong["alt_texts"] += 1

    # Verifica si todos los elementos input en la página tienen algún tipo de etiqueta (1.3.1 H44)
    def check_input_labels(self):
        # obtiene todos los elementos input y label
        input_elements = self.page.find_all("input")
        label_elements = self.page.find_all("label")
        for input_element in input_elements:
            # excluye elemento input de tipo hidden, submit, reset y button
            if ("type" in input_element.attrs and not input_element['type'] == "hidden" and not input_element['type'] == "submit" \
                    and not input_element['type'] == "reset" and not input_element['type'] == "button") or "type" not in input_element.attrs:
                # verifica si input es de tipo image y tiene un texto alternativo que no esté vacío
                if "type" in input_element.attrs and input_element['type'] == "image" and "alt" in input_element.attrs \
                        and not input_element['alt'] == "":
                    #print("  Input of type image labelled with alt text", Utils.xpath_soup(input_element))
                    self.correct["input_labels"] += 1
                # verifica si el elemento input usa aria-label
                elif "aria-label" in input_element.attrs and not input_element['aria-label'] == "":
                    self.correct["input_labels"] += 1
                # verifica si el elemento input usa aria-labelledby
                elif "aria-labelledby" in input_element.attrs and not input_element['aria-labelledby'] == "":
                    label_element = self.page.find(id=input_element['aria-labelledby'])
                    if not label_element is None:
                        texts_in_label_element = label_element.findAll(string=True)
                        if not texts_in_label_element == []:
                            self.correct["input_labels"] += 1
                        else:
                            self.wrong["input_labels"] += 1
                    else:
                        self.wrong["input_labels"] += 1
                else:
                    # verifica si el elemento input tiene un elemento label correspondiente
                    label_correct = False
                    for label_element in label_elements:
                        # verifica si el atributo "for" del elemento label es idéntico al "id" del elemento input
                        if "for" in label_element.attrs and "id" in input_element.attrs and label_element['for'] == input_element['id']:
                            label_correct = True
                    if label_correct:
                        self.correct["input_labels"] += 1
                    else:
                        self.wrong["input_labels"] += 1

    # Verifica si todos los elementos button e input de los tipos submit, button, y reset tienen algún tipo de contenido (1.1.1 y 2.4.4)
    def check_buttons(self):
        # obtiene todos los elementos button e input de tipos submit, button y reset
        input_elements = self.page.find_all("input", type=["submit", "button", "reset"])
        button_elements = self.page.find_all("button")

        for input_element in input_elements:
            # verifica si el elemento input tiene un atributo value que no esté vacío
            if "value" in input_element.attrs and not input_element['value'] == "":
                self.correct["empty_buttons"] += 1
            else:
                self.wrong["empty_buttons"] += 1

        for button_element in button_elements:
            # verifica si el button tiene contenido o un título
            texts = button_element.findAll(string=True)
            if not texts == [] or ("title" in button_element.attrs and not button_element["title"] == ""):
                self.correct["empty_buttons"] += 1
            else:
                self.wrong["empty_buttons"] += 1
    
    # Verifica si todos los enlaces de la página tienen algún tipo de contenido (2.4.4 G91 y H30)
    def check_links(self):
        # obtiene todos los elementos a
        link_elements = self.page.find_all("a")
        for link_element in link_elements:
            # verifica si el enlace tiene contenido
            texts_in_link_element = link_element.findAll(string=True)
            img_elements = link_element.findChildren("img", recursive=False)
            all_alt_texts_set = True
            for img_element in img_elements:
                alt_text = img_element.get_attribute_list('alt')[0]
                if alt_text is None or alt_text == "":
                    all_alt_texts_set = False
            if not texts_in_link_element == [] or (not img_elements == [] and all_alt_texts_set):
                self.correct["empty_links"] += 1
            else:
                self.wrong["empty_links"] += 1

    # Verifica si todos los textos de la página tienen un contraste suficientemente alto con el color del fondo (1.4.3 G18 y G145 (y 148))
    def check_color_contrast(self):
        # excluye script, estilo, título y elementos vacíos, así como doctype y comentarios
        texts_on_page = Utils.extract_texts(self.page)
        input_elements = self.page.find_all("input")
        elements_with_text = texts_on_page + input_elements
        for text in elements_with_text:
            try:
                selenium_element = self.driver.find_element(by="xpath", value=Utils.xpath_soup(text))
                # excluye textos invisibles
                element_visible = selenium_element.value_of_css_property('display')
                if not element_visible == "none" and (not text.name == "input" or (text.name == "input" \
                        and "type" in text.attrs and not text['type'] == "hidden")):
                    text_color = Utils.convert_to_rgba_value(selenium_element.value_of_css_property('color'))
                    background_color = Utils.get_background_color(self.driver, text)
    
                    # calcula contraste entre color del texto y color de fondo
                    contrast = Utils.get_contrast_ratio(eval(text_color[4:]), eval(background_color[4:]))
    
                    # obtiene el tamaño y el grosor de la fuente
                    font_size = selenium_element.value_of_css_property('font-size')
                    font_weight = selenium_element.value_of_css_property('font-weight')
    
                    if not font_size is None and font_size.__contains__("px") and \
                            (int(''.join(filter(str.isdigit, font_size))) >= 18 or ((font_weight == "bold" or font_weight == "700" \
                            or font_weight == "800" or font_weight == "900" or text.name == "strong") \
                            and int(''.join(filter(str.isdigit, font_size))) >= 14)):
                        if contrast >= 3:
                            self.correct["color_contrast"] += 1
                        else:
                            self.wrong["color_contrast"] += 1
                    else:
                        if contrast >= 4.5:
                            self.correct["color_contrast"] += 1
                        else:
                            self.wrong["color_contrast"] += 1
            except:
                continue

    # Calcula el resultado del test
    def calculate_result(self):
        # calcula métricas globales de correct y wrong
        correct = sum(self.correct.values())
        wrong = sum(self.wrong.values())
        ratio = correct / (correct + wrong)
        
        # verifica si ratio alcanza el valor mínimo deseado
        clase = 'sin_prob_aw' if ratio >= self.required_degree else 'con_prob_aw'

        stats = {f'nombre': self.nombre}
        stats.update({f'url': self.url})
        stats.update({f'{key}_ok': self.correct[key] for key in self.correct})
        stats.update({f'{key}_fail': self.wrong[key] for key in self.wrong})
        stats.update({f'ratio': ratio})
        stats.update({f'clase': clase})
        
        return pd.DataFrame([stats])

In [69]:
TAW('Accessible University', 'https://www.washington.edu/accesscomputing/AU/before.html').test_page()

,nombre,url,doc_language_ok,alt_texts_ok,input_labels_ok,empty_buttons_ok,empty_links_ok,color_contrast_ok,doc_language_fail,alt_texts_fail,input_labels_fail,empty_buttons_fail,empty_links_fail,color_contrast_fail,ratio,clase
0,Accessible University,https://www.washington.edu/accesscomputing/AU/...,0,6,0,3,29,113,1,5,10,0,0,19,0.811828,sin_prob_aw


In [70]:
TAW('Accessible University', 'https://www.washington.edu/accesscomputing/AU/after.html').test_page()

,nombre,url,doc_language_ok,alt_texts_ok,input_labels_ok,empty_buttons_ok,empty_links_ok,color_contrast_ok,doc_language_fail,alt_texts_fail,input_labels_fail,empty_buttons_fail,empty_links_fail,color_contrast_fail,ratio,clase
0,Accessible University,https://www.washington.edu/accesscomputing/AU/...,1,6,10,8,25,143,0,0,0,1,0,1,0.989744,sin_prob_aw


In [71]:
TAW('UNO', 'https://www.uno.edu.ar').test_page()

,nombre,url,doc_language_ok,alt_texts_ok,input_labels_ok,empty_buttons_ok,empty_links_ok,color_contrast_ok,doc_language_fail,alt_texts_fail,input_labels_fail,empty_buttons_fail,empty_links_fail,color_contrast_fail,ratio,clase
0,UNO,https://www.uno.edu.ar,1,18,1,0,274,177,0,0,0,1,1,160,0.744076,con_prob_aw


In [72]:
TAW('UNQ', 'https://www.unq.edu.ar/').test_page()

,nombre,url,doc_language_ok,alt_texts_ok,input_labels_ok,empty_buttons_ok,empty_links_ok,color_contrast_ok,doc_language_fail,alt_texts_fail,input_labels_fail,empty_buttons_fail,empty_links_fail,color_contrast_fail,ratio,clase
0,UNQ,https://www.unq.edu.ar/,1,9,2,8,137,178,0,12,1,3,28,43,0.793839,sin_prob_aw


In [73]:
df = pd.read_csv('universidades-nombre-url-code.csv', sep=';')
df.head()

,Institución,URL,status_code
0,Instituto Universitario de Gendarmería Nacional,https://iugna.edu.ar,200.0
1,Instituto Universitario de la Policía Federal ...,https://www.iupfa.edu.ar,200.0
2,Instituto Universitario de Seguridad Marítima,https://iusm.edu.ar,200.0
3,Universidad Autónoma de Entre Ríos,https://www.uader.edu.ar/,200.0
4,Universidad de Buenos Aires,https://www.uba.ar/,200.0


In [64]:
# Lista para acumular resultados parciales
resultados = []

for _, row in df.iterrows():
        df_parcial = TAW(row['Institución'], row['URL']).test_page()
        resultados.append(df_parcial)

# Concatenar todos los DataFrames en uno solo
df_total = pd.concat(resultados, ignore_index=True)

In [41]:
df_total.to_csv('universidades-taw.csv', sep=';', index=False)